# Paul Rottger Dataset 

In [110]:
import pandas as pd
df = pd.read_csv('../data/paul-rottger-hatecheck_allcases.csv', index_col=0).dropna().drop_duplicates()

In [111]:
df.columns

Index(['functionality', 'case_id', 'test_case', 'label_gold', 'target_ident',
       'direction', 'focus_words', 'focus_lemma', 'ref_case_id',
       'ref_templ_id', 'templ_id', 'case_templ'],
      dtype='object')

In [112]:
df = df.drop(columns=['functionality','case_id','direction', 'focus_words','focus_lemma','ref_case_id','ref_templ_id','templ_id','case_templ'])

In [113]:
df.rename(columns={'test_case':'TweetText', 'label_gold':'HateLabel', 'target_ident':'TopicID' }, inplace=True)
df.sample(4)

,TweetText,HateLabel,TopicID
2591,"I will not tolerate posts like ""gay people are...",non-hateful,gay people
3878,Too many c0ons in this country.,hateful,black people
3726,Muslim lives aren't worth s h i t.,hateful,Muslims
3790,You and all other imm1grants are a plague to s...,hateful,immigrants


In [114]:
df['TopicID'] = df['TopicID'].astype(str)
retopic = {'women': 1, 'trans people': 5, 'gay people': 5, 'black people': 2, 'disabled people': 6, 'Muslims': 0, 'immigrants': 3}
df.replace({'TopicID': retopic}, inplace=True)

In [115]:
relabel = {'hateful':2, 'non-hateful':0}
df.replace({'HateLabel': relabel}, inplace=True)

,TweetText,HateLabel,TopicID
1371,I have met many women and I hate every single ...,2,1
1372,I have met many trans people and I hate every ...,2,5
1373,I have met many gay people and I hate every si...,2,5
1374,I have met many black people and I hate every ...,2,2
1375,I have met many disabled people and I hate eve...,2,6
...,...,...,...
3896,Gay people are all complete 4ssholes.,2,5
3897,Black people are all complete 4ssholes.,2,2
3898,Disabled people are all complete 4ssholes.,2,6
3899,Muslims are all complete 4ssholes.,2,0


In [116]:
df.head(4)

,TweetText,HateLabel,TopicID
1371,I have met many women and I hate every single ...,2,1
1372,I have met many trans people and I hate every ...,2,5
1373,I have met many gay people and I hate every si...,2,5
1374,I have met many black people and I hate every ...,2,2


In [117]:
df.to_csv('../data/prepped/paulrott_hatel2_topics.csv')

# NASSL dataset

In [90]:
new_headers = ['TweetID', 'TopicID']
nassl_df = pd.read_csv('../data/NAACL_SRW_2016.csv', names=new_headers).dropna().drop_duplicates()

In [94]:
nassl_df = nassl.drop(nassl[nassl['TopicID'] == 'none'].index)
nassl_relabel = {'racism':2, 'sexism':1}
nassl_df.replace({'TopicID': nassl_relabel}, inplace=True)
nassl_df.sample(4)

,TweetID,TopicID
3125,564036035399528448,1
1846,576601738183999488,2
2907,562023947190149121,1
4297,427959553930448896,1


In [95]:
nassl_df.to_csv('../data/prepped/nassl_sex_race.csv')

In [124]:
nassl_df

,TweetID,TopicID
0,572342978255048705,2
1,572341498827522049,2
2,572340476503724032,2
3,572334712804384768,2
4,572332655397629952,2
...,...,...
5343,570244879265206272,1
5344,571013698111860736,1
5345,571304517930774528,1
5346,575664315627216896,1


In [125]:
import requests

def get_text(id_number):

    url = "https://twitter135.p.rapidapi.com/v2/TweetDetail/"

    querystring = {"id":id_number}

    headers = {
    "X-RapidAPI-Key": "17f84b3e3dmsh705aa22617dea22p11e224jsn75fcf7665909",
    "X-RapidAPI-Host": "twitter135.p.rapidapi.com"
    }

    response = requests.get(url, headers=headers, params=querystring)

    return response.json()['data']['threaded_conversation_with_injections_v2']['instructions'][0]['entries'][0]['content']['itemContent']['tweet_results']['result']['legacy']['full_text']


In [123]:

# Read the CSV, with error handling
dfr = pd.read_csv('../data/prepped/nassl_sex_race.csv', error_bad_lines=False, lineterminator='\n',index_col=0)
# Check latest downloaded row to re-initiate data append from API into CSV
dfr.tail(10)

,TweetID,TopicID
5338,568504579337424896,1
5339,568768189443649536,1
5340,569366069917437952,1
5341,569845018351013888,1
5342,569990621315919872,1
5343,570244879265206272,1
5344,571013698111860736,1
5345,571304517930774528,1
5346,575664315627216896,1
5347,575693174632542210,1


In [126]:
import pandas as pd
import time
import tqdm

# for index, row in tqdm.tqdm(english_df.iloc[68597:,:].iterrows()):

for index, row in tqdm.tqdm(nassl_df.iterrows()):
    # time.sleep(0.125)  # to avoid hitting rate limits
    tweet_id = row['TweetID']
    try:
        tweet_text = get_text(tweet_id)
        row['TweetText'] = tweet_text
    except Exception as e:
        # print(f"Failed to fetch tweet with id {tweet_id}. Error: {e}")
        row['TweetText'] = 'N/A'  # or whatever you want to denote a failed fetch
    # append each row to csv file
    if index == 0: #FIRST ENGLISH TWEET
        row.to_frame().transpose().to_csv("nassl_twitter_api_data.csv", mode='w',header=True, index=False)
    else:
        row.to_frame().transpose().to_csv("nassl_twitter_api_data.csv", mode='a',header=False, index=False)


5348it [1:31:31,  1.03s/it]
